<a href="https://colab.research.google.com/github/RutikaSansaria17/Quantification-of-Golgi-Dispersal-and-Classification-Using-Machine-Learning-Models/blob/main/quantification_of_golgi_dispersal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from skimage.io import imread, imshow
from skimage import morphology, filters, io, segmentation, color
from skimage.util import img_as_bool
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from skimage.segmentation import watershed
from skimage.measure import label, regionprops
from scipy import ndimage
import math
from google.colab.patches import cv2_imshow
import cv2
from scipy.ndimage import distance_transform_edt
from shapely.geometry import Polygon
from skimage import measure
from google.colab import files
import pandas as pd
import os
import glob
from pathlib import Path
from google.colab.patches import cv2_imshow

from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import skeletonize
from scipy import ndimage as ndi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install git+https://github.com/fish-quant/big-fish

In [ ]:
import bigfish.segmentation as segment
import bigfish.multistack as multistack
import os

import bigfish
import bigfish.stack as stack
import bigfish.plot as plot
print("Big-FISH version: {0}".format(bigfish.__version__))

In [ ]:
def func (img,image_file):

  physical_size =0.16049  #in um
  # Print result
  print(f"Physical dimensions: {width_um:.2f} x {height_um:.2f} µm")
  physical_size = width_um / img.shape[0]    # assuming square pixels
  print(f"Physical size of pixel: {physical_size}")
  print(f"Type of image: {img.dtype}")

  # Converting to gray-scale image
  image = img[:,:,1]

  # Denoising image
  # bigfish.stack.remove_background_mean approximates the background of the image
  # (the low frequency noise) by a large mean filter and substracts it from the original
  # image. It is only available for 2D images.
  image_background_mean = stack.remove_background_mean(image, kernel_shape="square", kernel_size=50)
  images = [image, image_background_mean]
  titles = ["Original image", "Removed mean background"]
  plot.plot_images(images, contrast=True, titles=titles)

    # Apply Gaussian filter to theimage to correct the background
  bkg_correction = filters.gaussian(image_background_mean, sigma=221)

  # Divide the image by the background correction and normalize
  corrected_cell = np.divide(image_background_mean, bkg_correction/bkg_correction.max())

  # Apply Otsu threshold to the corrected image to segment foreground and background
  threshold = filters.threshold_otsu(corrected_cell)
  ostu_mask = corrected_cell > threshold

  # Display the original and corrected images side by side
  f4, a4 = plt.subplots(1,2, figsize=(20,10))
  a41 = a4[0].imshow(image_background_mean, cmap='jet')
  plt.colorbar(a41, ax=a4[0],fraction=0.05, pad=0.05)
  a42 = a4[1].imshow(ostu_mask, cmap='jet')
  plt.colorbar(a42, ax=a4[1],fraction=0.05, pad=0.05)

  # Watershed segmentation

  # Now we want to separate the objects in image
  # Generate the markers as local maxima of the distance to the background
  distance = ndi.distance_transform_edt(ostu_mask)
  coords = peak_local_max(distance, footprint=np.ones((3, 3)), labels=ostu_mask)
  mask = np.zeros(distance.shape, dtype=bool)
  mask[tuple(coords.T)] = True
  markers, _ = ndi.label(mask)
  labels = watershed(-distance, markers, mask=ostu_mask)

  # Find the bounding boxes of each labeled object
  objects = ndi.find_objects(labels)

  # Extract each object as a separate array using NumPy indexing
  separated_objects = [ostu_mask[obj] for obj in objects]

  fig, axes = plt.subplots(ncols=4, figsize=(9, 3), sharex=True, sharey=True)
  ax = axes.ravel()

  ax[0].imshow(ostu_mask, cmap=plt.cm.gray)
  ax[0].set_title('Overlapping objects')
  ax[1].imshow(-distance, cmap=plt.cm.gray)
  ax[1].set_title('Distances')
  ax[2].imshow(labels, cmap=plt.cm.nipy_spectral)
  ax[2].set_title('Separated objects')
  ax[3].imshow(image)
  ax[3].set_title(ax[3].set_title(image_file.split("/")[-1]))


  for a in ax:
      a.set_axis_off()

  fig.tight_layout()
  plt.show()

  # # Display the separated objects
  fig, axes = plt.subplots(ncols=len(separated_objects), figsize=(len(separated_objects)*3, 3), sharex=True, sharey=True)
  for i, ax in enumerate(axes):
      ax.imshow(separated_objects[i], cmap=plt.cm.gray)
      ax.set_title(f'Object {i+1}')

  for ax in axes:
      ax.set_axis_off()

  fig.tight_layout()
  plt.show()

    # Get the region properties
  props = regionprops(labels)
  len(props)

  # Calculate total area and number of objects
  total_area = np.sum([prop.area for prop in props])
  num_objects = len(props)

  # Initialize variables for average orientation, perimeter, and eccentricity
  avg_orientation = 0
  avg_perimeter = 0
  avg_eccentricity = 0

  # Print the area, percentage area, and centroid coordinates of each region
  for i, prop in enumerate(props):
      area = prop.area
      percent_area = area / total_area * 100
      centroid = prop.centroid
      orientation = prop.orientation
      perimeter = prop.perimeter
      eccentricity = prop.eccentricity
      # Add orientation, perimeter, and eccentricity to the running totals
      avg_orientation += orientation
      avg_perimeter += perimeter
      avg_eccentricity += eccentricity

      # print(f"Object {i+1}: Area={area}, Percent Area={percent_area:.2f}%, Centroid={centroid}, Orientation={orientation:.2f}, Perimeter={perimeter:.2f}, Eccentricity={eccentricity:.2f}")


  # Calculate average area and total percentage area
  avg_area2 = total_area / num_objects
  avg_area1 = avg_area2 * (physical_size**2)
  avg_area ="{:.2f}".format(avg_area1)
  total_percent_area1 = total_area / (ostu_mask.shape[0] * ostu_mask.shape[1]) * 100
  total_percent_area ="{:.2f}".format(total_percent_area1)

  # Calculate average orientation, perimeter, and eccentricity
  avg_orientation /= num_objects
  avg_perimeter /= num_objects
  avg_eccentricity /= num_objects
  avg_perimeter2 = avg_perimeter * physical_size
  avg_orientation1 ="{:.2f}".format(avg_orientation)
  avg_perimeter1 ="{:.2f}".format(avg_perimeter2)
  avg_eccentricity1 ="{:.2f}".format(avg_eccentricity)


  # Print the total number of objects, average area, and total percentage area
  print(f"Total number of objects: {num_objects}")
  print(f"Average area: {avg_area}")
  print(f"Total percentage area: {total_percent_area}%")
  print(f"Average orientation: {avg_orientation1}")
  print(f"Average perimeter: {avg_perimeter1}")
  print(f"Average eccentricity: {avg_eccentricity1}")



  data["Image"].append(image_file.split("/")[-1])
  data["Average Area"].append(avg_area)
  data["Number of Objects"].append(num_objects)
  data["Total Percentage Area"].append(total_percent_area)
  data["Average Perimeter"].append(avg_perimeter1)
  data["Average Eccentricity"].append(avg_eccentricity1)
  data["Average Orientation"].append(avg_orientation1)

   # Load the image
  img = ostu_mask

  # Perform watershed segmentation to separate the objects
  distance = ndi.distance_transform_edt(img)
  local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)), labels=img)
  markers = ndi.label(local_maxi)[0]
  labels = watershed(-distance, markers, mask=img)

  # Measure the properties of the labeled objects
  props = measure.regionprops(labels)

  # Define the subtypes
  subtypes = {
      'loop': [],
      'globule': [],
      'lumps': [],
      'short': [],
      'medium_length': [],
      'long': [],
      'branch': []
  }

  # Initialize subtype areas
  loop_area = 0
  globule_area = 0
  lumps_area = 0
  short_area = 0
  medium_length_area = 0
  long_area = 0
  branch_area = 0

  # Calculate the properties of each object and classify it into a subtype
  for prop in props:
      # Calculate the axial ratio of the object
      if prop.minor_axis_length == 0:
          axial_ratio = 1
      else:
          axial_ratio = prop.major_axis_length / prop.minor_axis_length

      # Calculate the number of holes in the object
      objhole = len(measure.regionprops(np.logical_xor(prop.image.astype(np.uint8), prop.convex_image.astype(np.uint8)).astype(np.int))[1:])

      # Calculate the area of the object
      area = prop.area

      # Get the skeletonized image of the object
      skel = skeletonize(prop.image)

      # Calculate the maximum pixel length of the skeletonized object
      BrMaxLength = cv2.arcLength(max(cv2.findContours(skel.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0], key=cv2.contourArea), closed=True)

      # Calculate the number of skeletal branches of the Golgi-derived membrane structures
      branches = cv2.connectedComponents(skel.astype(np.uint8))[0] - 1
      BrNum = max(branches, 0)

      # Classify the object into a subtype
      if objhole == 1:
          subtypes['loop'].append(prop)
          loop_area += area
      elif area <= 30 and objhole == 0:
          subtypes['globule'].append(prop)
          globule_area += area
      elif area > 30 and axial_ratio <= 2 and objhole == 0:
          subtypes['lumps'].append(prop)
          lumps_area += area
      elif area > 30 and axial_ratio > 2 and objhole == 0 and BrNum == 1 and BrMaxLength <= 35:
          subtypes['short'].append(prop)
          short_area += area
      elif area > 30 and axial_ratio > 2 and objhole == 0 and BrNum == 1 and 35 < BrMaxLength < 70:
          subtypes['medium_length'].append(prop)
          medium_length_area += area
      elif area > 30 and axial_ratio > 2 and objhole == 0 and BrNum > 1:
          subtypes['branch'].append(prop)
          branch_area += area

  # Calculate the total area of each subtype in um^2
  loop_area_um = loop_area * physical_size * physical_size
  globule_area_um = globule_area * physical_size * physical_size
  lumps_area_um = lumps_area * physical_size * physical_size
  short_area_um = short_area * physical_size * physical_size
  medium_length_area_um = medium_length_area * physical_size * physical_size
  long_area_um = long_area * physical_size * physical_size
  branch_area_um = branch_area * physical_size * physical_size

  # Calculate the total area of all subtypes
  total_area = loop_area + globule_area + lumps_area + short_area + medium_length_area + long_area + branch_area
  total_area_um = total_area * physical_size * physical_size

  # Calculate the percentage of each subtype
  loop_percent = loop_area / total_area * 100
  globule_percent = globule_area / total_area * 100
  lumps_percent = lumps_area / total_area * 100
  short_percent = short_area / total_area * 100
  medium_length_percent = medium_length_area / total_area * 100
  long_percent = long_area / total_area * 100
  branch_percent = branch_area / total_area * 100

  loop_area_um1 ="{:.2f}".format(loop_area_um)
  loop_len = len(subtypes['loop'])
  loop_percent1 = "{:.2f}".format(loop_percent)

  globule_area_um1 = "{:.2f}".format(globule_area_um)
  globule_len =len(subtypes['globule'])
  globule_percent1 = "{:.2f}".format(globule_percent)

  lumps_area_um1 ="{:.2f}".format(lumps_area_um)
  lumps_len =len(subtypes['lumps'])
  lumps_percent1 = "{:.2f}".format(lumps_percent)

  short_area_um1 ="{:.2f}".format(short_area_um)
  short_len =len(subtypes['short'])
  short_percent1 = "{:.2f}".format(short_percent)

  medium_length_area_um1 ="{:.2f}".format(medium_length_area_um)
  medium_length_len =len(subtypes['medium_length'])
  medium_length_percent1 = "{:.2f}".format(medium_length_percent)

  long_area_um1 ="{:.2f}".format(long_area_um)
  long_len =len(subtypes['long'])
  long_percent1 = "{:.2f}".format(long_percent)

  branch_area_um1 ="{:.2f}".format(branch_area_um)
  branch_len =len(subtypes['branch'])
  branch_percent1 = "{:.2f}".format(branch_percent)

  total_area_um1 = "{:.2f}".format(total_area_um)



  # Print the results
  print(f"Loop subtype: {len(subtypes['loop'])} objects, {loop_percent:.2f}% of total area ({loop_area_um:.2f} um^2)")
  print(f"Globule subtype: {len(subtypes['globule'])} objects, {globule_percent:.2f}% of total area ({globule_area_um:.2f} um^2)")
  print(f"Lumps subtype: {len(subtypes['lumps'])} objects, {lumps_percent:.2f}% of total area ({lumps_area_um:.2f} um^2)")
  print(f"Short subtype: {len(subtypes['short'])} objects, {short_percent:.2f}% of total area ({short_area_um:.2f} um^2)")
  print(f"Medium length subtype: {len(subtypes['medium_length'])} objects, {medium_length_percent:.2f}% of total area ({medium_length_area_um:.2f} um^2)")
  print(f"Long subtype: {len(subtypes['long'])} objects, {long_percent:.2f}% of total area ({long_area_um:.2f} um^2)")
  print(f"Branch subtype: {len(subtypes['branch'])} objects, {branch_percent:.2f}% of total area ({branch_area_um:.2f} um^2)")
  print(f"Total area: {total_area} pixels, {total_area_um:.2f} um^2")


  data["Loop area"].append(loop_area_um1)
  data["Loop subtype"].append(loop_len)
  data[ "Loop percent"].append(loop_percent1)

  data["globule area"].append(globule_area_um1)
  data["globule subtype"].append(globule_len)
  data["globule  percent"].append(globule_percent1)

  data["lumps area"].append(lumps_area_um1)
  data["lumps subtype"].append(lumps_len)
  data["lumps percent"].append(lumps_percent1)

  data["short area"].append(short_area_um1)
  data["short subtype"].append(short_len)
  data["short percent"].append(short_percent1)

  data["medium_length area"].append(medium_length_area_um1)
  data["medium_length subtype"].append(medium_length_len)
  data["medium_length percent"].append(medium_length_percent1)

  data["long area"].append(long_area_um1)
  data["long subtype"].append(long_len)
  data["long percent"].append(long_percent1)

  data["branch area"].append(branch_area_um1)
  data["branch subtype"].append(branch_len)
  data["branch  percent"].append(branch_percent1)

  data["Total subtypes area"].append(total_area_um1)



In [ ]:
def split(img,n,image_file):
# n is mo. of cells in image
  green= img[:,:,1]
  threshold = green.max() * 0.1
  mask1 = green > threshold
  image1 = mask1
  # Threshold image to create binary mask
  mask2 = image1 > 0.5


  # Label connected regions in binary mask
  label_image2 = measure.label(mask2)

  # Get region properties
  props = measure.regionprops(label_image2)

  # Find indices of n regions with largest area

  max_areas = [0] * n
  max_indices = [0] * n
  for i, prop in enumerate(props):
      area = prop.area
      for j in range(n):
          if area > max_areas[j]:
              max_areas[j+1:n] = max_areas[j:n-1]
              max_indices[j+1:n] = max_indices[j:n-1]
              max_areas[j] = area
              max_indices[j] = i
              break

  # Create output images with n largest regions as one and all others as zero
  output_images = [np.zeros_like(image1) for _ in range(n)]
  for i in range(n):
      output_images[i][label_image2 == max_indices[i]+1] = 1

  # Display output images

  # Create a subplot with 1 row and n+1 columns
  fig, axes = plt.subplots(nrows=1, ncols=n+1, figsize=(10, 5))

  # Display the original image in the first column
  axes[0].imshow(img)
  axes[0].set_title('Original')

  # Display the output images in the remaining columns
  for i in range(n):
      axes[i+1].imshow(output_images[i])
      axes[i+1].set_title(f'Region {i+1}')

  # Show the plot
  plt.show()

  for i in np.arange(n):
    output_images[i] = output_images[i]*green



  for i in output_images:
    i = color.gray2rgb(i)
    func(i,image_file)

  return output_images





In [ ]:
  # Create a dictionary to store the data
data = {"Image": [], "Number of Objects": [], "Average Area": [], "Average Perimeter": [], "Average Eccentricity": [], "Average Orientation": [], "Total Percentage Area": [], "Loop area" : [], "Loop subtype" : [], "Loop percent" : [], "globule area" : [], "globule subtype" : [], "globule  percent" : [], "lumps area" : [], "lumps subtype" : [], "lumps percent" : [], "short area" : [], "short subtype" : [], "short percent" : [], "medium_length area" : [], "medium_length subtype" : [], "medium_length percent" : [], "long area" : [], "long subtype": [], "long percent" : [], "branch area" : [],  "branch subtype": [], "branch  percent" : [], "Total subtypes area" : [] }


In [ ]:
image_files = glob.glob(os.path.join('path', '*.tif'))

In [ ]:
# Process each image in the directory
for image_file in image_files:
  r = imread(image_file)
  split(r,2,image_file) #taken n=2 as example
  # Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(data)

In [ ]:
# Specify the path to the desired folder in your Google Drive
path = "path"


# Check if the folder exists, and create it if it doesn't
if not os.path.exists(path):
    os.makedirs(path)

# Save the DataFrame to an Excel sheet in the specified folder
filename = "result.xlsx"
filepath = os.path.join(path, filename)
df.to_excel(filepath, index=False)

In [ ]:
# Maximum Dispersion Distance
def max (img,image_file):

  physical_size = 0.16049  #in um
  # Print result
  print(f"Physical dimensions: {width_um:.2f} x {height_um:.2f} µm")
  physical_size = width_um / img.shape[0]    # assuming square pixels
  print(f"Physical size of pixel: {physical_size}")
  print(f"Type of image: {img.dtype}")

  # Converting to gray-scale image
  image = img[:,:,1]


  image_background_mean = stack.remove_background_mean(image, kernel_shape="square", kernel_size=50)


    # Apply Gaussian filter to theimage to correct the background
  bkg_correction = filters.gaussian(image_background_mean, sigma=221)

  # Divide the image by the background correction and normalize
  corrected_cell = np.divide(image_background_mean, bkg_correction/bkg_correction.max())

  # Apply Otsu threshold to the corrected image to segment foreground and background
  threshold = filters.threshold_otsu(corrected_cell)
  ostu_mask = corrected_cell > threshold



  # Watershed segmentation

  # Now we want to separate the objects in image
  # Generate the markers as local maxima of the distance to the background
  distance = ndi.distance_transform_edt(ostu_mask)
  coords = peak_local_max(distance, footprint=np.ones((3, 3)), labels=ostu_mask)
  mask = np.zeros(distance.shape, dtype=bool)
  mask[tuple(coords.T)] = True
  markers, _ = ndi.label(mask)
  labels = watershed(-distance, markers, mask=ostu_mask)

  # Find the bounding boxes of each labeled object
  objects = ndi.find_objects(labels)

  # Extract each object as a separate array using NumPy indexing
  separated_objects = [ostu_mask[obj] for obj in objects]


    # Get the region properties
  props = regionprops(labels)
  len(props)

  # Calculate total area and number of objects
  total_area = np.sum([prop.area for prop in props])
  num_objects = len(props)

  # Initialize variables for average orientation, perimeter, and eccentricity
  avg_orientation = 0
  avg_perimeter = 0
  avg_eccentricity = 0

  # Print the area, percentage area, and centroid coordinates of each region
  for i, prop in enumerate(props):
      area = prop.area
      percent_area = area / total_area * 100
      centroid = prop.centroid
      orientation = prop.orientation
      perimeter = prop.perimeter
      eccentricity = prop.eccentricity
      # Add orientation, perimeter, and eccentricity to the running totals
      avg_orientation += orientation
      avg_perimeter += perimeter
      avg_eccentricity += eccentricity

      # print(f"Object {i+1}: Area={area}, Percent Area={percent_area:.2f}%, Centroid={centroid}, Orientation={orientation:.2f}, Perimeter={perimeter:.2f}, Eccentricity={eccentricity:.2f}")



  # Initialize a list to store the centroid coordinates of each object
  centroids = []

  # Print the centroid coordinates of each object

  for i, prop in enumerate(props):
      centroid = prop.centroid
      centroids.append(centroid)

  # Masking
  cell = img[:,:,1]
  image_bool = cell > 25
  # images = [cell,image_bool]
  # titles = ["image","masked image"]
  # plot.plot_images(images, rescale=True, titles=titles)

  # GETTING NUCELUS
  # Invert the image
  inverted_image = 1 - image_bool

  # Apply clear boundary to make the background zero
  cleared_image = segmentation.clear_border(inverted_image)

  # fill holes
  filled_image = ndimage.binary_fill_holes(cleared_image)

  # FInding largest object
    # Threshold image to create binary mask
  binary_mask = filled_image > 0.5

  # Label connected regions in binary mask
  label_image = measure.label(binary_mask)

  # Get region properties
  props = measure.regionprops(label_image)

  # Find index of largest region
  max_area = 0
  max_index = 0
  for i, prop in enumerate(props):
      if prop.area > max_area:
          max_area = prop.area
          max_index = i

  # Create output image with largest region as one and all others as zero
  output_image = np.zeros_like(cell)
  output_image[label_image == max_index+1] = 1

  # # Display output image
  # io.imshow(output_image, cmap='gray')
  # io.show()

  # converting bool to 8 bit
  nucleus = output_image.astype(np.uint8)
  # plt.imshow(nucleus)

  # FINDING NUCLEUS CENTER
    # Find the indices of all non-zero elements in the image
  indices = np.nonzero(nucleus)

  # Calculate the centroid
  centroid = np.mean(indices, axis=1)

  # Mark the centroid on the image
  nucleus[int(centroid[0]), int(centroid[1])] = 2

  # Plot the image with centroid marked in red
  fig, ax = plt.subplots()
  ax.imshow(nucleus, cmap='gray')
  ax.scatter(centroid[1], centroid[0], c='r')
  ax.set_axis_off()
  plt.show()



  Mark the centroid on the image
  ostu_mask[int(centroid[0]), int(centroid[1])] = 2

  # Plot the image with centroid marked in red
  fig, ax = plt.subplots()
  ax.imshow(ostu_mask, cmap='gray')
  ax.scatter(centroid[0], centroid[1], c='r')
  ax.set_axis_off()
  plt.show()

  # FINDING FARTHEST PUNCTA
  x=[]
  y=[]
  for i in range(ostu_mask.shape[0]):
    for j in range(ostu_mask.shape[0]):
      if ostu_mask[i,j]==1:
        x.append(i)
        y.append(j)
  x1=np.array(x)
  y1=np.array(y)
  i=0
  centre=[centroid[0],centroid[1]]
  ListDistance=[]
  for _ in range(int(x1.shape[0]/2)):
    point=[x1[i],y1[i]]
    distance= math.dist(point,centre)

    ListDistance.append(distance)
    i+=1
  max = ListDistance.index(np.array(ListDistance).max())

  max_x = x1[max]
  max_y = y1[max]
  max_dispersion_length_pixel = ListDistance[max]
    # Convert max dispersion length from pixel units to micrometers
  pixel_size = physical_size # µm/pixel
  max_dispersion1 = max_dispersion_length_pixel * pixel_size
  max_dispersion ="{:.2f}".format(max_dispersion1)

  # print(f"{max_dispersion_length_pixel} pixels is maximum dispersion length & coordinates are {max_x}, {max_y}")
  # print(f"{max_dispersion} µm is maximum dispersion length in micrometers")


  # Drawing maximum dispersion line
  cell_image =(ostu_mask.astype(np.uint8) * 255) #converting to 8-bit
  # Define points
  pt1 = (max_y,max_x)
  pt2 = (int(centroid[1]),int(centroid[0]))

  # Draw line
  color = (255) # BGR format
  thickness = 2
  cv2.line(cell_image, pt1, pt2, color, thickness)

  # Display image
  cv2_imshow(cell_image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()
    # Populate the dictionary with data

  data["Image"].append(image_file.split("/")[-1])
  data["Maximum Dispersion"].append(max_dispersion)





In [ ]:
def split(img,n,image_file):

  green= img[:,:,1]
  threshold = green.max() * 0.1
  mask1 = green > threshold
  image1 = mask1
  # Threshold image to create binary mask
  mask2 = image1 > 0.5


  # Label connected regions in binary mask
  label_image2 = measure.label(mask2)

  # Get region properties
  props = measure.regionprops(label_image2)

  # Find indices of n regions with largest area

  max_areas = [0] * n
  max_indices = [0] * n
  for i, prop in enumerate(props):
      area = prop.area
      for j in range(n):
          if area > max_areas[j]:
              max_areas[j+1:n] = max_areas[j:n-1]
              max_indices[j+1:n] = max_indices[j:n-1]
              max_areas[j] = area
              max_indices[j] = i
              break

  # Create output images with n largest regions as one and all others as zero
  output_images = [np.zeros_like(image1) for _ in range(n)]
  for i in range(n):
      output_images[i][label_image2 == max_indices[i]+1] = 1

  # Display output images

  # Create a subplot with 1 row and n+1 columns
  fig, axes = plt.subplots(nrows=1, ncols=n+1, figsize=(10, 5))

  # Display the original image in the first column
  axes[0].imshow(img)
  axes[0].set_title(image_file.split("/")[-1])

  # Display the output images in the remaining columns
  for i in range(n):
      axes[i+1].imshow(output_images[i])
      axes[i+1].set_title(f'I {i+1}')

  # Show the plot
  plt.show()

  for i in np.arange(n):
    output_images[i] = output_images[i]*green



  for i in output_images:
    i = color.gray2rgb(i)
    max(i,image_file)

  return output_images

In [ ]:
data = {"Image": [], "Maximum Dispersion": [] }

In [ ]:
# Process each image in the directory
for image_file in image_files:
  try:
    r = imread(image_file)
    split(r,2,image_file) #taken n=2 as example
    pass
  except Exception as e:
        print(f"Error in iteration {image_file}: {e}")
        continue
  # Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(data)

In [ ]:
# Specify the path to the desired folder in your Google Drive
path = "path"


# Check if the folder exists, and create it if it doesn't
if not os.path.exists(path):
    os.makedirs(path)

# Save the DataFrame to an Excel sheet in the specified folder
filename = "result.xlsx"
filepath = os.path.join(path, filename)
df.to_excel(filepath, index=False)